<a href="https://colab.research.google.com/github/caiocesarcosta/ai-job-assistant/blob/main/ai_job_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
import numpy as np

### Passo 2: Criando o Conjunto de Dados ###
perguntas = [
    "Qual sua principal habilidade?",
    "Conte-me sobre sua experiência em [área de atuação]",
    "Quais seus objetivos profissionais?",
]

respostas = [
    "Minha principal habilidade é o desenvolvimento de aplicações Android inovadoras e eficientes, sempre buscando as melhores soluções para os desafios propostos.",
    "Tenho 8 anos de experiência em desenvolvimento Android, atuando em projetos desafiadores como o desenvolvimento de aplicativos para automação residencial na Flex Automation e a criação e atualização de diversos projetos do Itaú na BRQ, incluindo o SuperApp, Itaú One e o programa de pontos Iupp.",
    "Meus objetivos profissionais são continuar a crescer na área de desenvolvimento mobile, explorando novas tecnologias e metodologias, e contribuir para a criação de aplicativos que facilitem a vida das pessoas."
]

### Pré-processamento dos dados ###
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(perguntas + respostas)

perguntas_sequencias = tokenizer.texts_to_sequences(perguntas)
respostas_sequencias = tokenizer.texts_to_sequences(respostas)

perguntas_treino = tf.keras.preprocessing.sequence.pad_sequences(perguntas_sequencias)
respostas_treino = tf.keras.preprocessing.sequence.pad_sequences(respostas_sequencias)

### Criando o Modelo ###
modelo = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64)  # Sem ativação para similaridade de cosseno
])

### Função de Similaridade de Cosseno ###
def similaridade_cosseno(y_true, y_pred):
    y_true = tf.nn.l2_normalize(y_true, axis=1)
    y_pred = tf.nn.l2_normalize(y_pred, axis=1)
    return -tf.reduce_sum(y_true * y_pred, axis=1)

modelo.compile(optimizer='adam', loss=similaridade_cosseno)

### Criando embeddings para as perguntas de treinamento ###
embeddings_perguntas_treino = modelo.predict(perguntas_treino)

### Treinando o Modelo ###
modelo.fit(perguntas_treino, embeddings_perguntas_treino, epochs=100)

### Função para Encontrar a Resposta Mais Similar ###
def encontrar_resposta(pergunta):
    pergunta_sequencia = tokenizer.texts_to_sequences([pergunta])
    pergunta_processada = tf.keras.preprocessing.sequence.pad_sequences(pergunta_sequencia,
                                                                        maxlen=perguntas_treino.shape[1])
    embeddings_perguntas = modelo.predict(perguntas_treino)
    embedding_pergunta = modelo.predict(pergunta_processada)
    similaridades = tf.keras.losses.cosine_similarity(embedding_pergunta, embeddings_perguntas).numpy()
    indice_resposta = np.argmax(similaridades)
    return respostas[indice_resposta]

### Testando o Chatbot ###
while True:
    pergunta = input("Digite sua pergunta: ")
    if pergunta == "sair":
        break
    resposta = encontrar_resposta(pergunta)
    print("Bot:", resposta)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: -1.0000
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: -1.0000
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: -0.9620
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: -0.9948
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: -0.9853
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: -0.9825
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: -0.9890
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: -0.9940
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - loss: -0.9951
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: -0.9945
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: -0.9941
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - loss: -0.9946
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - loss: -0.9959
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: -0.9971
Epoch 15/100
1/1 ━━━━━━━━━━━━

KeyboardInterrupt: Interrupted by user